In [1]:
'''
Limits for personal use: requests/min is 5, tokens/day is 300k
'''

import os
from crewai_tools.tools import WebsiteSearchTool
from dotenv import load_dotenv
load_dotenv()

web_search_tool = WebsiteSearchTool()

In [6]:
from textwrap import dedent
from crewai import Agent
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
  model="claude-3-sonnet-20240229",
  verbose=True,
  temperature=0.3,
  anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"))


## Agents
class MeetingPreparationAgents():

  def research_agent(self):
    return Agent(
        role='Research Specialist',
        goal=
        'Conduct thorough research on people and companies involved in the meeting',
        tools=[web_search_tool],
        backstory=dedent("""\
          As a Research Specialist, your mission is to uncover detailed information
          about the individuals and entities participating in the meeting. Your insights
          will lay the groundwork for strategic meeting preparation."""),
        llm=llm,
        max_rpm=2,
        verbose=True,
        memory=True)

  def industry_analysis_agent(self):
    return Agent(
        role='Tech Analyst',
        goal=
        'Analyze the current tech trends, challenges, and opportunities',
        tools=[web_search_tool],
        backstory=dedent("""\
          As a Tech Analyst, your analysis will identify key trends,
          challenges facing the tech, and potential opportunities that
          could be leveraged during the meeting for strategic advantage."""),
        llm=llm,
        max_rpm=3,
        verbose=True,
        memory=True)

  def meeting_strategy_agent(self):
    return Agent(
        role='Meeting Strategy Advisor',
        goal=
        'Develop talking points, questions, and strategic angles for the meeting',
        tools=[web_search_tool],
        backstory=dedent("""\
          As a Strategy Advisor, your expertise will guide the development of
          talking points, insightful questions, and strategic angles
          to ensure the meeting's objectives are achieved."""),
        llm=llm,
        max_rpm=1,
        verbose=True,
        memory=True)

  def summary_and_briefing_agent(self):
    return Agent(
        role='Briefing Coordinator',
        goal=
        'Compile all gathered information into a concise, informative briefing document',
        tools=[web_search_tool],
        backstory=dedent("""\
          As the Briefing Coordinator, your role is to consolidate the research,
          analysis, and strategic insights."""),
        llm=llm,
        max_rpm=1,
        verbose=True,
        memory=True)



In [7]:
## Tasks
from crewai import Task


class MeetingPreparationTasks():

  def research_task(self, agent, participants, context):
    return Task(description=dedent(f"""\
        Conduct comprehensive research on each of the individuals involved in the upcoming meeting. Gather information on recent news, achievements, professional background, and any relevant business activities. Today is April 3, 2024.

        Participants: {participants}
        Meeting Context: {context}"""),
                expected_output=dedent("""\
        A detailed report summarizing key findings about each participant, highlighting information that could be relevant for the meeting."""
                                       ),
                async_execution=True,
                agent=agent)

  def industry_analysis_task(self, agent, participants, context):
    return Task(description=dedent(f"""\
        Analyze the current tech trends, challenges, and opportunities
        relevant to the meeting's context. Consider market reports, recent
        developments, and expert opinions to provide a comprehensive
        overview of the tech landscape. Today is April 3, 2024.

        Participants: {participants}
        Meeting Context: {context}"""),
                expected_output=dedent("""\
        An insightful analysis that identifies major trends, potential
        challenges, and strategic opportunities."""),
                async_execution=True,
                agent=agent)

  def meeting_strategy_task(self, agent, context, objective):
    return Task(description=dedent(f"""\
        Develop strategic talking points, questions, and discussion angles
        for the meeting based on the research and industry analysis conducted

        Meeting Context: {context}
        Meeting Objective: {objective}"""),
                expected_output=dedent("""\
        Complete report with a list of key talking points, strategic questions
        to ask to help achieve the meetings objective during the meeting."""),
                agent=agent)

  def summary_and_briefing_task(self, agent, context, objective):
    return Task(description=dedent(f"""\
        Compile all the research findings, industry analysis, and strategic
        talking points into a concise, comprehensive briefing document for
        the meeting.
        Ensure the briefing is easy to digest and equips the meeting
        participants with all necessary information and strategies.

        Meeting Context: {context}
        Meeting Objective: {objective}"""),
                expected_output=dedent("""\
        A well-structured briefing document that includes sections for
        participant bios, industry overview, talking points, and
        strategic recommendations."""),
                agent=agent)

In [8]:
## Crew
from crewai import Crew

tasks = MeetingPreparationTasks()
agents = MeetingPreparationAgents()

participants = "Andrew Ng, Andrej Karpathy"
context = "Comptetitive analysis of 'AI Agents' technology"
objective = "Convince participants that the company should purchase Agents of Tomorrow, which is a startup that created a promising framework for orchestrating AI Agents"

# Create Agents
researcher_agent = agents.research_agent()
industry_analyst_agent = agents.industry_analysis_agent()
meeting_strategy_agent = agents.meeting_strategy_agent()
summary_and_briefing_agent = agents.summary_and_briefing_agent()

# Create Tasks
research = tasks.research_task(researcher_agent, participants, context)
industry_analysis = tasks.industry_analysis_task(industry_analyst_agent,
                                                 participants, context)
meeting_strategy = tasks.meeting_strategy_task(meeting_strategy_agent, context,
                                               objective)
summary_and_briefing = tasks.summary_and_briefing_task(
    summary_and_briefing_agent, context, objective)

meeting_strategy.context = [research, industry_analysis]
summary_and_briefing.context = [research, industry_analysis, meeting_strategy]

In [9]:
# Create Crew responsible for Copy
crew = Crew(agents=[
    researcher_agent, industry_analyst_agent, meeting_strategy_agent,
    summary_and_briefing_agent
],
            tasks=[
                research, industry_analysis, meeting_strategy,
                summary_and_briefing
            ])

print("Kickoff!")
result = crew.kickoff()

# Print results
print("\n\n################################################")
print("## Here is the result")
print("################################################\n")
print(result)

2024-04-03 05:07:56,200 - 28664 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


Kickoff!


> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
Thought: To conduct thorough research on the meeting participants, I will need to gather information from reputable sources about their backgrounds, recent news, achievements, and relevant business activities. I should start by searching for their names and the company/organization they are associated with.

Action: Search in a specific website
Action Input: {"search_query": "Andrew Ng", "website": "https://en.wikipedia.org"}
Doc content has not changed. Skipping creating chunks and embeddings
Thought: To provide a comprehensive analysis of current tech trends, challenges, and opportunities related to 'AI Agents' technology, I will need to gather relevant information from reliable sources. I should start by searching for recent market reports, expert opinions, and news articles on this topic.

Action: Search in a specific website
Action Input: {
    "search_query": "AI agents technology tre

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.58it/s]


Successfully saved https://www.google.com/search?tbm=nws (DataType.WEB_PAGE). New chunks count: 2
 

Relevant Content:
Andrew Ng - Wikipedia Jump to content From Wikipedia, the free encyclopedia American artificial intelligence researcher For the physicist, see Andrew Ng (physicist). Andrew NgNg in 2017BornAndrew Yan-Tak Ng (1976-04-18) April 18, 1976 (age 47)[2][citation needed]United Kingdom[2]NationalityBritishEducationRaffles InstitutionAlma materUniversity of California, Berkeley (PhD)Massachusetts Institute of Technology (MS)Carnegie Mellon University (BS)Known forArtificial intelligence, deep learning, MOOC, education technologySpouse Carol E. Reiley ​(m. 2014)​Children2Awards 2007 Sloan Fellowship 2008 MIT Technology Review TR35 2009 IJCAI Computers and Thought Award 2013 Time 100 Most Influential People 2013 Fortune's 40 under 40 2013 CNN 10 2014 Fast Company's Most Creative People in Business 2015 World Economic Forum Young Global Leaders 2023 Time 100 Most Influential People

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.40it/s]


...

Successfully saved https://www.forbes.com/ (DataType.WEB_PAGE). New chunks count: 2
 

Relevant Content:
DeepLearning.AI: Start or Advance Your Career in AIAI is the new electricity. You are the spark.Get the latest AI news, courses, events, and insights from Andrew Ng and other AI leaders. Join over 7 million people learning how to use and build AI AI Courses and SpecializationsBuild a foundation of machine learning and AI skills, and understand how to apply them in the real world. prevnextIn Collaboration WithprevnextThe largest weekly AI newsletterWhat matters in AI right nowMar 27, 2024One Agent For Many Worlds, Cross-Species Cell Embeddings, U.S. Deploys AI Targeting, Robo Football Gets RealThe Batch AI News and Insights: Last week, I described four design patterns for AI agentic workflows that I believe will drive significant progress this year.Mar 20, 2024Robots Talk Back, AI Security Risks, Political Deepfakes, Pretrained Models on the CheapThe Batch AI News and Insights: I thi

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


Successfully saved https://www.techcrunch.com/ (DataType.WEB_PAGE). New chunks count: 3
 

Relevant Content:
TechCrunch | Startup and Technology News Anthropic researchers wear down AI ethics with repeated questions Devin Coldewey Rubrik’s IPO filing hints at thawing public markets for tech companies Alex Wilhelm, Rebecca Szkutak, Marina Temkin Meta (again) denies that Netflix read users’ private Facebook messages Sarah Perez Nicki Minaj, Billie Eilish, Katy Perry and other musicians sign letter against irresponsible AI Amanda Silberling AWS unveils new service for cloud-based rendering projects Kyle Wiggers The Latest Chatbots have come a long way. For years, they were limited to responding with predetermined replies that followed a simple logic structure. But customers can have complex problems, and no tree-diagra For years, the solar energy sector has grappled with interseasonal energy storage. The ability to harness the surplus solar energy of summer months for use during the winte

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.44it/s]


Successfully saved https://venturebeat.com (DataType.WEB_PAGE). New chunks count: 1
 

Relevant Content:
OpenAI may be dreaming of $100B 5GW AI 'Stargate' supercomputer Comment Play it again, Sam What if AI produces code not just quickly but also, dunno, securely, DARPA wonders As 70% of boffinry nerve center's projects involve machine learning AI + ML02 Apr 2024 | 5 OWASP server blunder exposes decade of resumes Irony alerts: Open Web Application Security Project Foundation suffers lapse Security02 Apr 2024 | 3 UK and US to jointly develop AI test suites to tackle risks Memorandum of Understanding penned to put models, systems, and agents through their paces AI + ML02 Apr 2024 | 3 Datacenter outages are on the decline, but when they hit, they hit hard Power snafus take limelight in latest downtime diary from Uptime Institute On-Prem02 Apr 2024 | 2 Pandabuy confirms crooks nabbed data on 1.3M punters Nothing says 'sorry' like 10 percent off shipping for a month Cyber-crime02 Apr 2024 |

In [11]:
from IPython.display import display, Markdown
display(Markdown(result))


AI Agent Technology Briefing

I. Industry Overview

The field of AI agent technology has seen rapid advancement driven by breakthroughs in machine learning, natural language processing, computer vision, and reinforcement learning. Major players include tech giants like Google, Microsoft, OpenAI, DeepMind as well as numerous startups.

Key Trends:
- Agentic AI systems capable of perception, decision-making and action
- Multi-agent systems to tackle complex problems through coordination  
- Human-AI interaction models for seamless collaboration
- Generative AI agents powered by large language models
- AI safety and ethics to ensure alignment with human values

II. Agents of Tomorrow Positioning 

Agents of Tomorrow is an emerging startup that has developed a modular framework for composing and orchestrating teams of specialized AI agents.

Potential Differentiators:
- Modular, composable architecture for multi-agent development
- Extensive library of pre-trained agents for common tasks
- Tools for training custom agents on domain data
- Robust coordination protocols and human-AI interfaces  
- Emphasis on transparency, interpretability and ethical AI

While an innovative approach, Agents of Tomorrow faces intense competition from major tech companies investing heavily in AI agent R&D.

III. Technical Assessment

Detailed technical specifications or third-party evaluations of Agents of Tomorrow's framework are not publicly available. A comprehensive technical analysis is limited without access to additional documentation from the company.

Key Considerations:
- Request in-depth technical briefing and product demo from Agents of Tomorrow
- Assess innovative capabilities compared to other AI agent offerings
- Understand approach to machine perception, decision-making, environment interaction
- Identify potential technical limitations, risks or hurdles
- Review case studies, benchmarks or performance validation

IV. Strategic Recommendations  

1. Pursue opportunity to acquire Agents of Tomorrow if their technology demonstrates clear competitive advantages and strategic value.

2. Prioritize technical due diligence and risk assessment through direct engagement with the Agents of Tomorrow team.

3. Develop concrete integration plans for effectively leveraging their AI agent capabilities across business operations if acquired.

4. Invest in AI ethics practices to ensure safe and responsible deployment aligning with human values as agent technologies advance.

5. Explore partnership opportunities with other key players to accelerate AI agent innovation and adoption.

The AI agent field represents an exciting frontier with potential to drive immense innovation. Agents of Tomorrow's unique orchestration framework is intriguing but requires deeper validation. Balancing the risks and potential rewards will be crucial as we evaluate this opportunity.